# 🔬 V8.0 Concept Proof: Spatial Noise Forensics
## Three Tests to Validate the New Forensic Hypothesis

**Previous finding:** HLL temporal flicker energy does NOT separate real from fake.  
Fakes are actually quieter than real videos (0.97x — basically identical).

**New hypothesis:**  
> The forensic signal is *spatial*, not temporal.  
> A deepfake has a **sharp noise boundary** where the generated face meets the background.  
> This boundary is **persistent** across frames — unlike natural texture variation which is random.

### Three Tests
| Test | Signal | Validates |
|------|--------|-----------|
| 1 | Noise gradient sharpness (90th pct / median) | Blend boundary exists |
| 2 | Frame-to-frame noise correlation | Boundary is persistent (not random) |
| 3 | Spatial autocorrelation break | Noise field is discontinuous |

### Go/No-Go Criteria
- **Green light:** Signal separates real/fake on both FF++ c23 AND Celeb-DF → Build V8.0
- **Amber:** Signal works on Celeb-DF but not FF++ c23 → Need compression robustness module  
- **Red:** Signal fails on both → Different forensic anchor needed

## Section 1 — Setup

In [ ]:
import os, json, random, warnings
from pathlib import Path
from typing import Optional, Dict, List, Tuple
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats
from scipy.signal import correlate2d
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

OUTPUT_DIR = Path('/kaggle/working/v8_concept')
PLOTS_DIR  = OUTPUT_DIR / 'plots'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
PLOTS_DIR.mkdir(exist_ok=True)

print(f"NumPy {np.__version__}, OpenCV {cv2.__version__}")
print(f"Outputs → {OUTPUT_DIR}")


## Section 2 — Core Forensic Tools

### The Noise Extraction Strategy
We suppress image content and reveal processing artifacts using three complementary filters:

1. **Gaussian residual** — simplest: `noise = frame - GaussianBlur(frame)`
2. **Laplacian** — second derivative, maximally sensitive to sharp edges in the noise field  
3. **SRM high-pass** — Steganalysis Rich Model filter, specifically designed to reveal 
   manipulation artifacts by canceling natural image gradients

All three are applied and compared — if the signal appears in all three, it's robust.

In [ ]:
# ─── SRM-style high-pass filters ─────────────────────────────────────────────
# These 5×5 kernels are from the SRM paper (Fridrich & Kodovsky 2012)
# They are designed to suppress image content and amplify manipulation residuals

SRM_FILTERS = {
    'srm1': np.array([
        [ 0,  0, 0,  0, 0],
        [ 0, -1, 2, -1, 0],
        [ 0,  2,-4,  2, 0],
        [ 0, -1, 2, -1, 0],
        [ 0,  0, 0,  0, 0]], dtype=np.float32) / 4.0,

    'srm2': np.array([
        [-1,  2, -2,  2, -1],
        [ 2, -6,  8, -6,  2],
        [-2,  8,-12,  8, -2],
        [ 2, -6,  8, -6,  2],
        [-1,  2, -2,  2, -1]], dtype=np.float32) / 12.0,

    'edge': np.array([
        [ 0,  0, -1,  0,  0],
        [ 0, -1,  2, -1,  0],
        [-1,  2,  0,  2, -1],
        [ 0, -1,  2, -1,  0],
        [ 0,  0, -1,  0,  0]], dtype=np.float32) / 2.0,
}

LAPLACIAN = np.array([[0, 1, 0],[1,-4, 1],[0, 1, 0]], dtype=np.float32)


def extract_noise_residual(frame_gray: np.ndarray,
                            method: str = 'gaussian') -> np.ndarray:
    """
    Extract high-frequency noise residual from a single grayscale frame.
    
    Args:
        frame_gray: (H, W) float32 in [0, 1]
        method: 'gaussian' | 'laplacian' | 'srm1' | 'srm2' | 'edge'
    Returns:
        residual: (H, W) float32 — noise field
    """
    if method == 'gaussian':
        blurred = cv2.GaussianBlur(frame_gray, (5, 5), 0)
        return frame_gray - blurred

    elif method == 'laplacian':
        lap = cv2.filter2D(frame_gray, -1, LAPLACIAN)
        return lap

    elif method in SRM_FILTERS:
        return cv2.filter2D(frame_gray, -1, SRM_FILTERS[method])

    else:
        raise ValueError(f"Unknown method: {method}")


def frames_to_gray(frames_rgb: np.ndarray) -> np.ndarray:
    """Convert (T, H, W, 3) uint8 to (T, H, W) float32 in [0,1]."""
    return (0.299 * frames_rgb[..., 0].astype(np.float32) +
            0.587 * frames_rgb[..., 1].astype(np.float32) +
            0.114 * frames_rgb[..., 2].astype(np.float32)) / 255.0


# ─── Self-test ────────────────────────────────────────────────────────────────
_test_frame = np.random.rand(112, 112).astype(np.float32)
for m in ['gaussian', 'laplacian', 'srm1', 'srm2', 'edge']:
    r = extract_noise_residual(_test_frame, m)
    print(f"  {m:12s}: residual shape={r.shape}, std={r.std():.5f}")
print("✅ Noise extraction tools ready")


In [ ]:
# ─── TEST 1: Noise Gradient Sharpness ────────────────────────────────────────
def noise_gradient_sharpness(frames_rgb: np.ndarray,
                              noise_method: str = 'gaussian') -> Dict:
    """
    Measure internal sharpness of the noise field.
    
    Key metric: 90th_percentile / median of gradient magnitude.
    
    Real face: noise field is smooth, gradients are small and uniform.
               ratio ≈ 2-3x (natural variation)
    Deepfake:  noise field has a sharp ring at the blend boundary.
               ratio >> 3x (sharp peak against smooth background)
    
    Returns dict with scalar metrics per video.
    """
    gray = frames_to_gray(frames_rgb)           # (T, H, W)
    T = gray.shape[0]
    
    sharpness_ratios = []
    p90_vals, median_vals = [], []
    gradient_maps = []
    
    for t in range(T):
        noise = extract_noise_residual(gray[t], noise_method)
        # Gradient magnitude of the noise field
        gx = cv2.Sobel(noise, cv2.CV_32F, 1, 0, ksize=3)
        gy = cv2.Sobel(noise, cv2.CV_32F, 0, 1, ksize=3)
        grad_mag = np.sqrt(gx**2 + gy**2)
        
        p90    = np.percentile(grad_mag, 90)
        median = np.median(grad_mag)
        ratio  = p90 / max(median, 1e-8)
        
        sharpness_ratios.append(ratio)
        p90_vals.append(p90)
        median_vals.append(median)
        gradient_maps.append(grad_mag)
    
    return {
        'sharpness_ratio_mean':   float(np.mean(sharpness_ratios)),
        'sharpness_ratio_median': float(np.median(sharpness_ratios)),
        'sharpness_ratio_std':    float(np.std(sharpness_ratios)),
        'p90_mean':               float(np.mean(p90_vals)),
        'median_mean':            float(np.mean(median_vals)),
        'per_frame_ratios':       sharpness_ratios,
        'mean_gradient_map':      np.mean(gradient_maps, axis=0),  # (H, W)
    }


# ─── TEST 2: Temporal Persistence ────────────────────────────────────────────
def temporal_persistence(frames_rgb: np.ndarray,
                          noise_method: str = 'gaussian') -> Dict:
    """
    Measure frame-to-frame correlation of the noise residual field.
    
    Real: noise changes with content → low frame-to-frame correlation
    Fake: generator fingerprint is stable → HIGH frame-to-frame correlation
    
    Key metric: mean Pearson correlation between consecutive noise maps.
    """
    gray  = frames_to_gray(frames_rgb)
    T     = gray.shape[0]
    
    noise_maps = np.stack([extract_noise_residual(gray[t], noise_method)
                           for t in range(T)], axis=0)    # (T, H, W)
    
    # Frame-to-frame Pearson correlation of flattened noise maps
    consecutive_corrs = []
    for t in range(T - 1):
        n1 = noise_maps[t].ravel()
        n2 = noise_maps[t + 1].ravel()
        # Pearson correlation
        corr = np.corrcoef(n1, n2)[0, 1]
        consecutive_corrs.append(float(corr) if not np.isnan(corr) else 0.0)
    
    # Also measure: variance of the MEAN noise map
    # High variance in mean = noise is spatially structured (fake)
    # Low variance in mean  = noise averages to zero (real, stochastic)
    mean_noise_map = noise_maps.mean(axis=0)    # (H, W)
    mean_map_var   = float(np.var(mean_noise_map))
    
    # And: how much does the noise pattern change over time?
    # Compute std of each pixel across T, then average
    pixel_temporal_std = noise_maps.std(axis=0).mean()
    
    return {
        'consec_corr_mean':    float(np.mean(consecutive_corrs)),
        'consec_corr_median':  float(np.median(consecutive_corrs)),
        'consec_corr_std':     float(np.std(consecutive_corrs)),
        'mean_map_variance':   mean_map_var,
        'pixel_temporal_std':  float(pixel_temporal_std),
        'per_frame_corrs':     consecutive_corrs,
        'mean_noise_map':      mean_noise_map,    # (H, W)
    }


# ─── TEST 3: Spatial Autocorrelation ─────────────────────────────────────────
def spatial_autocorrelation(frames_rgb: np.ndarray,
                             noise_method: str = 'gaussian',
                             lags: List[int] = [1, 3, 5, 10, 20]) -> Dict:
    """
    Measure spatial autocorrelation of the noise field at multiple distances.
    
    Real face: noise is spatially correlated at short distances (camera sensor)
               correlation decays smoothly with distance.
    Deepfake:  noise field has a sharp break at the blend boundary.
               autocorrelation drops suddenly at the boundary distance.
    
    We measure: for each lag d, correlation between noise(x,y) and noise(x+d,y).
    A sharp drop in the autocorrelation profile indicates a spatial discontinuity.
    """
    gray = frames_to_gray(frames_rgb)
    T    = gray.shape[0]
    H, W = gray.shape[1], gray.shape[2]
    
    all_lag_corrs = {d: [] for d in lags}
    
    for t in range(T):
        noise = extract_noise_residual(gray[t], noise_method)
        noise_flat = noise - noise.mean()    # zero-mean
        
        for d in lags:
            if d >= W:
                continue
            # Horizontal lag-d correlation
            n1 = noise_flat[:, :-d].ravel()
            n2 = noise_flat[:,  d:].ravel()
            if len(n1) < 10:
                continue
            corr = np.corrcoef(n1, n2)[0, 1]
            if not np.isnan(corr):
                all_lag_corrs[d].append(float(corr))
    
    lag_means = {d: float(np.mean(v)) if v else 0.0
                 for d, v in all_lag_corrs.items()}
    
    # Autocorrelation decay rate: how fast does correlation drop with distance?
    lag_vals   = [lag_means[d] for d in lags]
    decay_rate = (lag_vals[0] - lag_vals[-1]) / max(lags[-1] - lags[0], 1)
    
    # Sharpness of decay: std of differences between consecutive lags
    diffs         = np.diff(lag_vals)
    decay_std     = float(np.std(diffs))
    
    return {
        'lag_means':    lag_means,
        'decay_rate':   decay_rate,
        'decay_std':    decay_std,   # high = irregular/sharp decay = boundary
        'lag_profile':  lag_vals,
    }


print("✅ All three test functions defined")


## Section 3 — Dataset Loading

In [ ]:
# ─── Dataset discovery ────────────────────────────────────────────────────────
KAGGLE_INPUT = Path('/kaggle/input')
N_SAMPLE     = 20
FACE_SIZE    = 112
N_FRAMES     = 32

def locate_root(base, known_path_parts, marker_dirs):
    """Find a dataset root by known path or by presence of marker subdirs."""
    known = base
    for part in known_path_parts:
        known = known / part
    if known.exists():
        return known
    for d in sorted(base.rglob('*')):
        if d.is_dir():
            hits = sum(1 for m in marker_dirs if (d / m).exists())
            if hits >= len(marker_dirs) // 2 + 1:
                return d
    return None

FF_METHODS = ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures',
              'FaceShifter', 'DeepFakeDetection']

FF_ROOT = locate_root(KAGGLE_INPUT,
    ['datasets','xdxd003','ff-c23','FaceForensics++_C23'],
    FF_METHODS)

CELEB_ROOT = locate_root(KAGGLE_INPUT,
    ['datasets','reubensuju','celeb-df-v2'],
    ['Celeb-real','Celeb-synthesis'])

print(f"FF++    root: {FF_ROOT}")
print(f"Celeb-DF root: {CELEB_ROOT}")

# ─── Build video lists ────────────────────────────────────────────────────────
FF_VIDEOS   = {}
CELEB_VIDEOS = {}

if FF_ROOT:
    real_paths = list(FF_ROOT.rglob('original*/*.mp4'))
    if not real_paths:
        real_paths = [p for p in FF_ROOT.rglob('*.mp4') if 'original' in str(p).lower()]
    FF_VIDEOS['real'] = sorted(real_paths)
    for m in FF_METHODS:
        paths = list((FF_ROOT / m).glob('*.mp4')) if (FF_ROOT / m).exists() else []
        if paths:
            FF_VIDEOS[m] = sorted(paths)
    for k, v in FF_VIDEOS.items():
        print(f"  FF++/{k:25s}: {len(v):4d} videos")

if CELEB_ROOT:
    CELEB_VIDEOS['real'] = (list((CELEB_ROOT / 'Celeb-real').glob('*.mp4')) +
                            list((CELEB_ROOT / 'YouTube-real').glob('*.mp4')))
    CELEB_VIDEOS['fake'] = list((CELEB_ROOT / 'Celeb-synthesis').glob('*.mp4'))
    for k, v in CELEB_VIDEOS.items():
        print(f"  Celeb-DF/{k:10s}: {len(v):4d} videos")


In [ ]:
# ─── Video loader ─────────────────────────────────────────────────────────────
def extract_frames(video_path: str, n_frames: int = N_FRAMES,
                   size: int = FACE_SIZE) -> Optional[np.ndarray]:
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total < n_frames:
        cap.release()
        return None
    indices = np.linspace(0, total - 1, n_frames, dtype=int)
    frames  = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Center crop to face region (rough but consistent)
        h, w = frame.shape[:2]
        margin = 0.10
        y1, y2 = int(h * margin), int(h * (1 - margin))
        x1, x2 = int(w * 0.15), int(w * 0.85)
        frame = frame[y1:y2, x1:x2]
        frame = cv2.resize(frame, (size, size))
        frames.append(frame)
    cap.release()
    if len(frames) < n_frames // 2:
        return None
    while len(frames) < n_frames:
        frames.append(frames[-1])
    return np.stack(frames[:n_frames], axis=0)


def sample(lst, n=N_SAMPLE, seed=SEED):
    rng = random.Random(seed)
    return rng.sample(lst, min(n, len(lst)))


# ─── Quick load test ──────────────────────────────────────────────────────────
if FF_VIDEOS.get('real'):
    test_frames = extract_frames(str(FF_VIDEOS['real'][0]))
    if test_frames is not None:
        print(f"✅ Load test OK — shape: {test_frames.shape}")
        r = noise_gradient_sharpness(test_frames)
        print(f"   Sharpness ratio: {r['sharpness_ratio_mean']:.3f}")
    else:
        print("⚠️  Load test failed")


## Section 4 — Run All Three Tests on FF++ c23

This is the primary experiment. We run all three tests on 20 videos per class.
Runtime: ~8-12 minutes on T4.

In [ ]:
# ─── Master analysis function ─────────────────────────────────────────────────
NOISE_METHOD = 'gaussian'   # primary; we'll compare all methods in Section 6

def analyze_class(video_paths: list, label: str,
                  n: int = N_SAMPLE, verbose: bool = True) -> List[Dict]:
    results = []
    sampled = sample(video_paths, n)
    
    for i, vpath in enumerate(sampled):
        frames = extract_frames(str(vpath))
        if frames is None:
            continue
        
        t1 = noise_gradient_sharpness(frames, NOISE_METHOD)
        t2 = temporal_persistence(frames, NOISE_METHOD)
        t3 = spatial_autocorrelation(frames, NOISE_METHOD)
        
        results.append({
            'label':  label,
            'video':  Path(vpath).name,
            # Test 1
            'sharpness_ratio':    t1['sharpness_ratio_mean'],
            'p90':                t1['p90_mean'],
            'per_frame_ratios':   t1['per_frame_ratios'],
            'mean_gradient_map':  t1['mean_gradient_map'],
            # Test 2
            'consec_corr':        t2['consec_corr_mean'],
            'mean_map_var':       t2['mean_map_variance'],
            'pixel_temporal_std': t2['pixel_temporal_std'],
            'per_frame_corrs':    t2['per_frame_corrs'],
            'mean_noise_map':     t2['mean_noise_map'],
            # Test 3
            'lag_profile':        t3['lag_profile'],
            'decay_std':          t3['decay_std'],
            'decay_rate':         t3['decay_rate'],
        })
        
        if verbose and (i + 1) % 5 == 0:
            r = results[-1]
            print(f"  [{label:15s}] {i+1:2d}/{n} | "
                  f"sharp={r['sharpness_ratio']:.3f} | "
                  f"corr={r['consec_corr']:.4f} | "
                  f"decay_std={r['decay_std']:.4f}")
    return results


# ─── Run on FF++ ──────────────────────────────────────────────────────────────
print("=" * 70)
print("RUNNING ON FF++ c23")
print("=" * 70)

FF_RESULTS = {}
run_order  = ['real'] + [m for m in ['Deepfakes','Face2Face','FaceSwap','NeuralTextures']
                          if m in FF_VIDEOS]

for label in run_order:
    print(f"\n[{label}]")
    FF_RESULTS[label] = analyze_class(FF_VIDEOS[label], label)
    print(f"  → {len(FF_RESULTS[label])} videos done")

print("\n✅ FF++ analysis complete")


## Section 5 — Results: FF++ c23

In [ ]:
# ─── Color scheme ─────────────────────────────────────────────────────────────
COLORS = {
    'real':           '#2ecc71',
    'Deepfakes':      '#e74c3c',
    'Face2Face':      '#e67e22',
    'FaceSwap':       '#9b59b6',
    'NeuralTextures': '#3498db',
}

def get_metric(results_dict, key):
    return {k: [r[key] for r in v] for k, v in results_dict.items()}

def boxplot_comparison(ax, data_dict, title, ylabel, colors=COLORS):
    groups = list(data_dict.keys())
    data   = [data_dict[g] for g in groups]
    cols   = [colors.get(g, '#95a5a6') for g in groups]
    bp = ax.boxplot(data, patch_artist=True,
                    medianprops=dict(color='black', linewidth=2.5),
                    notch=False)
    for patch, c in zip(bp['boxes'], cols):
        patch.set_facecolor(c)
        patch.set_alpha(0.75)
    ax.set_xticklabels([g.replace('NeuralTextures','NeuralTex.') for g in groups],
                       rotation=30, ha='right')
    ax.set_ylabel(ylabel)
    ax.set_title(title, fontweight='bold')
    ax.grid(True, alpha=0.3)
    # Annotate medians
    for i, (g, vals) in enumerate(zip(groups, data)):
        med = np.median(vals)
        ax.text(i+1, med, f'{med:.3f}', ha='center', va='bottom',
                fontsize=8, fontweight='bold')
    return bp


In [ ]:
# ─── Figure 1: Three metrics side by side ─────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('FF++ c23 — Three Forensic Signal Tests\n'
             'Does Spatial Noise Analysis Separate Real from Fake?',
             fontsize=14, fontweight='bold')

sharp  = get_metric(FF_RESULTS, 'sharpness_ratio')
corr   = get_metric(FF_RESULTS, 'consec_corr')
decay  = get_metric(FF_RESULTS, 'decay_std')

boxplot_comparison(axes[0], sharp, 
    'Test 1: Noise Gradient Sharpness\n(90th pct / median)',
    'Sharpness Ratio (higher = sharper internal peak)')
axes[0].set_xlabel('→ Higher for fakes if hypothesis holds')

boxplot_comparison(axes[1], corr,
    'Test 2: Temporal Persistence\n(frame-to-frame noise correlation)',
    'Consecutive Frame Noise Correlation')
axes[1].set_xlabel('→ Higher for fakes = persistent generator fingerprint')

boxplot_comparison(axes[2], decay,
    'Test 3: Spatial Autocorrelation Decay Std\n(irregularity of decay profile)',
    'Decay Irregularity (std of lag differences)')
axes[2].set_xlabel('→ Higher for fakes = sharp spatial discontinuity')

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'ff_three_tests.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved ff_three_tests.png")


In [ ]:
# ─── Statistical significance table ──────────────────────────────────────────
print("\n📊 STATISTICAL RESULTS — FF++ c23")
print("=" * 80)

metrics = [
    ('sharpness_ratio', 'Test 1: Gradient Sharpness'),
    ('consec_corr',     'Test 2: Temporal Persistence'),
    ('decay_std',       'Test 3: Autocorr Decay Std'),
]

for metric_key, metric_name in metrics:
    print(f"\n{metric_name}:")
    print(f"  {'Class':<18} {'Mean':>9} {'Median':>9} {'Std':>9} {'vs Real':>9} {'p-value':>10} {'Signal?':>10}")
    print(f"  {'-'*76}")
    
    real_vals = [r[metric_key] for r in FF_RESULTS.get('real', [])]
    real_med  = np.median(real_vals) if real_vals else 1.0
    
    for label, results in FF_RESULTS.items():
        vals   = [r[metric_key] for r in results]
        mean   = np.mean(vals)
        median = np.median(vals)
        std    = np.std(vals)
        ratio  = median / max(real_med, 1e-10)
        
        if label != 'real' and real_vals:
            _, p = stats.mannwhitneyu(real_vals, vals, alternative='two-sided')
            sig  = '✅ YES' if p < 0.05 else ('⚠️  WEAK' if p < 0.20 else '❌ NO')
        else:
            p, sig = 1.0, '—'
        
        print(f"  {label:<18} {mean:>9.4f} {median:>9.4f} {std:>9.4f} "
              f"{ratio:>9.2f}x {p:>10.4f} {sig:>10}")


In [ ]:
# ─── Figure 2: Mean gradient heatmaps real vs Deepfakes ───────────────────────
methods_to_show = ['real'] + [m for m in ['Deepfakes','Face2Face','FaceSwap','NeuralTextures']
                               if m in FF_RESULTS][:3]

fig, axes = plt.subplots(2, len(methods_to_show), figsize=(5*len(methods_to_show), 10))
fig.suptitle('Noise Gradient Heatmaps — Where is the Blend Boundary?\n'
             'Top: Mean gradient map | Bottom: Mean noise map',
             fontsize=13, fontweight='bold')

global_grad_max  = max(
    np.mean([r['mean_gradient_map'] for r in FF_RESULTS[m]], axis=0).max()
    for m in methods_to_show if FF_RESULTS.get(m))
global_noise_max = max(
    np.abs(np.mean([r['mean_noise_map'] for r in FF_RESULTS[m]], axis=0)).max()
    for m in methods_to_show if FF_RESULTS.get(m))

for col, method in enumerate(methods_to_show):
    results = FF_RESULTS.get(method, [])
    if not results:
        continue
    
    grad_maps  = np.stack([r['mean_gradient_map'] for r in results])
    noise_maps = np.stack([r['mean_noise_map']     for r in results])
    
    avg_grad  = grad_maps.mean(axis=0)
    avg_noise = noise_maps.mean(axis=0)
    
    color = COLORS.get(method, '#95a5a6')
    
    # Top row: gradient heatmap
    ax = axes[0, col]
    im = ax.imshow(avg_grad, cmap='hot', vmin=0, vmax=global_grad_max,
                   interpolation='bilinear')
    plt.colorbar(im, ax=ax, fraction=0.046)
    median_sharp = np.median([r['sharpness_ratio'] for r in results])
    ax.set_title(f'{method}\nSharpness={median_sharp:.3f}',
                 fontweight='bold',
                 bbox=dict(facecolor=color, alpha=0.7, boxstyle='round'),
                 color='white' if method != 'real' else 'black')
    ax.axis('off')
    
    # Bottom row: noise field
    ax = axes[1, col]
    im = ax.imshow(avg_noise, cmap='RdBu_r', 
                   vmin=-global_noise_max, vmax=global_noise_max,
                   interpolation='bilinear')
    plt.colorbar(im, ax=ax, fraction=0.046)
    median_corr = np.median([r['consec_corr'] for r in results])
    ax.set_title(f'Noise map\nCorr={median_corr:.4f}', fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'ff_heatmaps.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved ff_heatmaps.png")
print("\n💡 Key: if deepfake heatmaps show a ring/oval brighter than real → boundary signal confirmed")
print("         if all heatmaps look identical → signal doesn't survive c23 compression")


In [ ]:
# ─── Figure 3: Temporal trajectory of sharpness ───────────────────────────────
fig, axes = plt.subplots(1, len(methods_to_show), figsize=(5*len(methods_to_show), 5))
fig.suptitle('Test 2: Temporal Persistence — Frame-by-Frame Sharpness Ratio\n'
             'Real: random fluctuation | Fake: persistently elevated?',
             fontsize=13, fontweight='bold')

for col, method in enumerate(methods_to_show):
    ax  = axes[col]
    res = FF_RESULTS.get(method, [])[:8]
    color = COLORS.get(method, '#95a5a6')
    
    all_trajs = []
    for r in res:
        traj = np.array(r['per_frame_ratios'])
        ax.plot(traj, alpha=0.3, linewidth=1, color=color)
        all_trajs.append(traj)
    
    if all_trajs:
        min_len = min(len(t) for t in all_trajs)
        stacked = np.stack([t[:min_len] for t in all_trajs])
        mean_t  = stacked.mean(axis=0)
        std_t   = stacked.std(axis=0)
        x = np.arange(min_len)
        ax.plot(mean_t, color='black', linewidth=2.5, label='Mean')
        ax.fill_between(x, mean_t-std_t, mean_t+std_t, alpha=0.2, color='black')
        
        # Measure persistence: std of the trajectory (low = persistent, high = erratic)
        persistence = np.mean([np.std(t) for t in all_trajs])
        overall_mean = np.mean([np.mean(t) for t in all_trajs])
    else:
        persistence, overall_mean = 0, 0
    
    ax.set_title(f'{method}\nmean={overall_mean:.2f}, variation={persistence:.2f}',
                 color=color, fontweight='bold')
    ax.set_xlabel('Frame index')
    ax.set_ylabel('Sharpness ratio')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=8)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'ff_temporal_trajectory.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved ff_temporal_trajectory.png")


In [ ]:
# ─── Figure 4: Autocorrelation profiles ──────────────────────────────────────
LAGS = [1, 3, 5, 10, 20]

fig, ax = plt.subplots(figsize=(10, 6))
fig.suptitle('Test 3: Spatial Autocorrelation Profile\n'
             'Sharp drop = blend boundary; smooth decay = natural texture',
             fontsize=13, fontweight='bold')

for method, results in FF_RESULTS.items():
    color = COLORS.get(method, '#95a5a6')
    profiles = [r['lag_profile'] for r in results]
    min_len  = min(len(p) for p in profiles)
    profiles = [p[:min_len] for p in profiles]
    mean_profile = np.mean(profiles, axis=0)
    std_profile  = np.std(profiles, axis=0)
    x = LAGS[:min_len]
    
    ax.plot(x, mean_profile, 'o-', color=color, linewidth=2,
            markersize=6, label=method)
    ax.fill_between(x,
                    mean_profile - std_profile,
                    mean_profile + std_profile,
                    alpha=0.1, color=color)

ax.set_xlabel('Spatial lag (pixels)')
ax.set_ylabel('Spatial autocorrelation of noise field')
ax.set_title('Autocorrelation Decay by Class')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(0, color='black', linestyle='--', linewidth=0.8)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'ff_autocorrelation.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved ff_autocorrelation.png")


## Section 6 — Cross-Dataset Validation: Celeb-DF

**The critical test.** If the signal found in FF++ c23 also appears in Celeb-DF  
(different generation method, different compression), it's a universal forensic signal.

If it only appears in Celeb-DF but not FF++ → compression destroys the signal in c23  
→ V8.0 needs explicit compression robustness from day one.

In [ ]:
# ─── Run on Celeb-DF ─────────────────────────────────────────────────────────
CELEB_RESULTS = {}

if CELEB_VIDEOS:
    print("=" * 70)
    print("RUNNING ON CELEB-DF v2")
    print("=" * 70)
    for split in ['real', 'fake']:
        if CELEB_VIDEOS.get(split):
            print(f"\n[CelebDF_{split}]")
            CELEB_RESULTS[f'CelebDF_{split}'] = analyze_class(
                CELEB_VIDEOS[split], f'CelebDF_{split}')
            print(f"  → {len(CELEB_RESULTS[f'CelebDF_{split}'])} videos done")
else:
    print("⚠️  Celeb-DF not available — attach reubensuju/celeb-df-v2")


In [ ]:
# ─── Cross-dataset comparison plot ───────────────────────────────────────────
if CELEB_RESULTS:
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle('Cross-Dataset Validation — Does the Signal Transfer to Celeb-DF?\n'
                 'FF++ c23 (heavy compression) vs Celeb-DF (lighter compression)',
                 fontsize=13, fontweight='bold')
    
    CELEB_COLORS = {'CelebDF_real': '#27ae60', 'CelebDF_fake': '#c0392b'}
    all_colors   = {**COLORS, **CELEB_COLORS}
    
    # Combine FF++ real + Deepfakes + CelebDF for comparison
    compare = {
        'FF_real':    FF_RESULTS.get('real', []),
        'FF_Dfakes':  FF_RESULTS.get('Deepfakes', []),
        'CDF_real':   CELEB_RESULTS.get('CelebDF_real', []),
        'CDF_fake':   CELEB_RESULTS.get('CelebDF_fake', []),
    }
    compare_colors = {
        'FF_real':   COLORS['real'],
        'FF_Dfakes': COLORS['Deepfakes'],
        'CDF_real':  CELEB_COLORS['CelebDF_real'],
        'CDF_fake':  CELEB_COLORS['CelebDF_fake'],
    }
    
    for ax_idx, (metric_key, title, ylabel) in enumerate([
        ('sharpness_ratio', 'Test 1: Gradient Sharpness', 'Sharpness Ratio'),
        ('consec_corr',     'Test 2: Temporal Persistence', 'Frame Correlation'),
        ('decay_std',       'Test 3: Autocorr Decay Std', 'Decay Irregularity'),
    ]):
        ax   = axes[ax_idx]
        data_d = {k: [r[metric_key] for r in v] for k, v in compare.items() if v}
        boxplot_comparison(ax, data_d, title, ylabel, colors=compare_colors)
        
        # Draw vertical line between FF++ and CDF groups
        if len(data_d) > 2:
            ax.axvline(2.5, color='gray', linestyle='--', alpha=0.7)
            ax.text(1.5, ax.get_ylim()[1]*0.98, 'FF++', ha='center',
                    fontsize=9, color='gray')
            ax.text(3.5, ax.get_ylim()[1]*0.98, 'Celeb-DF', ha='center',
                    fontsize=9, color='gray')
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'cross_dataset_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ Saved cross_dataset_comparison.png")


## Section 7 — Which Noise Filter Works Best?

Testing all five noise extraction methods on 5 real vs 5 Deepfakes videos.  
We pick the best filter for V8.0's Stage 1 noise extraction.

In [ ]:
# ─── Quick filter comparison ──────────────────────────────────────────────────
FILTER_METHODS = ['gaussian', 'laplacian', 'srm1', 'srm2', 'edge']

def quick_filter_test(video_paths_real, video_paths_fake, n=5):
    results = {m: {'real': [], 'fake': []} for m in FILTER_METHODS}
    
    for label, vpaths in [('real', video_paths_real), ('fake', video_paths_fake)]:
        for vpath in sample(vpaths, n):
            frames = extract_frames(str(vpath))
            if frames is None:
                continue
            for method in FILTER_METHODS:
                r = noise_gradient_sharpness(frames, method)
                results[method][label].append(r['sharpness_ratio_mean'])
    return results


print("Testing all noise filters (sharpness ratio — higher = more signal)...")
print("─" * 65)

filter_results = None
if FF_VIDEOS.get('real') and FF_VIDEOS.get('Deepfakes'):
    filter_results = quick_filter_test(FF_VIDEOS['real'], FF_VIDEOS['Deepfakes'])
    
    print(f"{'Filter':<14} {'Real median':>12} {'Fake median':>12} {'Ratio':>8} {'p-value':>10}")
    print("─" * 60)
    
    best_method, best_ratio = 'gaussian', 0.0
    for method, data in filter_results.items():
        real_v = data['real']
        fake_v = data['fake']
        if not real_v or not fake_v:
            continue
        real_med = np.median(real_v)
        fake_med = np.median(fake_v)
        ratio = fake_med / max(real_med, 1e-8)
        _, p  = stats.mannwhitneyu(real_v, fake_v, alternative='two-sided') if len(real_v) > 1 else (0, 1.0)
        sig   = '✅' if p < 0.05 else ('⚠️' if p < 0.2 else '❌')
        print(f"{method:<14} {real_med:>12.4f} {fake_med:>12.4f} {ratio:>8.3f}x {p:>10.4f} {sig}")
        if abs(ratio - 1.0) > abs(best_ratio - 1.0):
            best_ratio, best_method = ratio, method
    
    print(f"\n→ Best filter: {best_method} (ratio={best_ratio:.3f}x)")
    print(f"  Recommendation for V8.0 Stage 1: use '{best_method}'")


## Section 8 — Summary & Go/No-Go Decision

In [ ]:
# ─── Auto-generate Go/No-Go report ───────────────────────────────────────────
print("=" * 70)
print("V8.0 CONCEPT PROOF — GO / NO-GO DECISION")
print("=" * 70)

results_summary = {}

for metric_key, test_name, direction in [
    ('sharpness_ratio', 'Test 1: Gradient Sharpness',    'higher_fake'),
    ('consec_corr',     'Test 2: Temporal Persistence',  'higher_fake'),
    ('decay_std',       'Test 3: Autocorr Decay Std',    'higher_fake'),
]:
    real_vals = [r[metric_key] for r in FF_RESULTS.get('real', [])]
    real_med  = np.median(real_vals) if real_vals else 0
    
    method_results = {}
    for label in ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']:
        vals = [r[metric_key] for r in FF_RESULTS.get(label, [])]
        if not vals:
            continue
        fake_med = np.median(vals)
        ratio    = fake_med / max(real_med, 1e-10)
        _, p     = stats.mannwhitneyu(real_vals, vals, alternative='two-sided') if real_vals else (0,1)
        method_results[label] = {'ratio': ratio, 'p': p}
    
    # Signal quality: how many methods separate significantly?
    n_sig     = sum(1 for r in method_results.values() if r['p'] < 0.05)
    n_methods = len(method_results)
    
    if n_sig >= 3:
        verdict = '🟢 STRONG SIGNAL'
    elif n_sig >= 1:
        verdict = '🟡 WEAK SIGNAL'
    else:
        verdict = '🔴 NO SIGNAL'
    
    results_summary[test_name] = {'verdict': verdict, 'n_sig': n_sig, 'n_total': n_methods}
    print(f"\n{test_name}")
    print(f"  Significant separation: {n_sig}/{n_methods} methods")
    print(f"  Verdict: {verdict}")

print("\n" + "=" * 70)
print("FINAL DECISION")
print("=" * 70)

green = sum(1 for v in results_summary.values() if '🟢' in v['verdict'])
yellow = sum(1 for v in results_summary.values() if '🟡' in v['verdict'])

if green >= 2:
    decision = "🟢 GO — Build V8.0 around spatial noise forensics"
    path     = "Primary: noise gradient sharpness + temporal persistence"
elif green >= 1 or yellow >= 2:
    decision = "🟡 CONDITIONAL GO — Signal exists but weak on FF++ c23"
    path     = "Need: compression robustness module in V8.0 Stage 1"
else:
    decision = "🔴 NO-GO — Spatial noise signal insufficient on FF++ c23"
    path     = "Need: test on Celeb-DF / raw FF++ / try identity drift"

print(f"\n{decision}")
print(f"Architecture path: {path}")

# Celeb-DF verdict
if CELEB_RESULTS:
    cdf_real = [r['sharpness_ratio'] for r in CELEB_RESULTS.get('CelebDF_real', [])]
    cdf_fake = [r['sharpness_ratio'] for r in CELEB_RESULTS.get('CelebDF_fake', [])]
    if cdf_real and cdf_fake:
        _, p_cdf = stats.mannwhitneyu(cdf_real, cdf_fake, alternative='two-sided')
        ratio_cdf = np.median(cdf_fake) / max(np.median(cdf_real), 1e-10)
        print(f"\nCeleb-DF sharpness ratio: {ratio_cdf:.3f}x (p={p_cdf:.4f})")
        if p_cdf < 0.05 and ratio_cdf > 1.1:
            print("🟢 Signal survives in Celeb-DF → universal, not codec-specific")
        elif p_cdf < 0.05:
            print("🟡 Weak signal in Celeb-DF")
        else:
            print("🔴 No signal in Celeb-DF either → different forensic anchor needed")


In [ ]:
# ─── Save all results ─────────────────────────────────────────────────────────
def to_json_safe(obj):
    if isinstance(obj, np.ndarray): return obj.tolist()
    if isinstance(obj, (np.float32, np.float64, np.float16)): return float(obj)
    if isinstance(obj, (np.int32, np.int64)): return int(obj)
    return obj

def summarize(results_dict):
    out = {}
    for label, results in results_dict.items():
        out[label] = {
            'sharpness_ratio': {
                'mean':   float(np.mean([r['sharpness_ratio'] for r in results])),
                'median': float(np.median([r['sharpness_ratio'] for r in results])),
                'std':    float(np.std([r['sharpness_ratio'] for r in results])),
            },
            'consec_corr': {
                'mean':   float(np.mean([r['consec_corr'] for r in results])),
                'median': float(np.median([r['consec_corr'] for r in results])),
            },
            'decay_std': {
                'mean':   float(np.mean([r['decay_std'] for r in results])),
                'median': float(np.median([r['decay_std'] for r in results])),
            },
        }
    return out

final_results = {
    'ff_plus_plus': summarize(FF_RESULTS),
    'celeb_df':     summarize(CELEB_RESULTS) if CELEB_RESULTS else {},
    'decision':     results_summary,
}

out_path = OUTPUT_DIR / 'v8_concept_results.json'
with open(out_path, 'w') as f:
    json.dump(final_results, f, indent=2, default=to_json_safe)

print(f"✅ Results saved → {out_path}")
print(f"✅ Plots saved   → {PLOTS_DIR}")
print("\n🏁 Concept proof complete. Share the JSON + plots to decide V8.0 architecture.")
